In [10]:
# read excell shet to df
import pandas as pd

# file_path = "./Data/Ocean Governance and ocean economy governance matrix_IGOs.xlsx"
data = pd.read_excel(path, sheet_name="Sheet1")

# backup
df = data.copy(deep=True)

FileNotFoundError: [Errno 2] No such file or directory: 'Data/Ocean Governance and ocean economy governance matrix_IGOs.xlsx'

In [1]:
import pandas as pd
import re
from collections import Counter

# Sample data (replace with your full dataset in a CSV file)
data = {
    "Institution": ["IOC", "FAO", "IMO"],  # Add all 52 institutions here
    "Practical Vertical Coordination": [
        "Vertical coordination within the IOC involves collaboration with various levels of institutions and authorities. The IOC, under UNESCO, engages with Member States, United Nations bodies, and other international organizations... (https://www.oceandecade.org/...)",
        "The FAO’s vertical coordination involves collaboration with various levels of governance and institutions, including the United Nations, specialized agencies, regional and intergovernmental organizations... (https://www.fao.org/...)",
        "Vertical coordination within IMO involves collaboration with higher and lower level institutions, such as the United Nations, other specialized agencies, regional and sub-regional organizations... (https://www.imo.org/...)"
    ]
}

# Step 1: Cleaning the Data
def clean_text(text):
    # Remove URLs
    text = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', text)
    # Correct common typos and standardize terms
    text = text.replace("United Nations", "UN").replace("member states", "Member States")
    # Simplify verbose phrases
    text = re.sub(r'which are responsible for implementing', 'tasked with implementing', text)
    return text.strip()

df = pd.DataFrame(data)
df["Cleaned Practical Vertical Coordination"] = df["Practical Vertical Coordination"].apply(clean_text)

# Step 2: Summarizing the Data
def summarize_text(text):
    # Simple rule-based summary: extract key entities and mechanisms
    summary = ""
    if "collaboration" in text.lower():
        summary += "Coordinates with "
        if "UN" in text: summary += "UN agencies, "
        if "Member States" in text: summary += "Member States, "
        summary = summary.rstrip(", ")
    if "Assembly" in text or "Council" in text or "Secretariat" in text:
        summary += " through its "
        if "Assembly" in text: summary += "Assembly, "
        if "Council" in text: summary += "Council, "
        if "Secretariat" in text: summary += "Secretariat, "
        summary = summary.rstrip(", ")
    if "regional" in text.lower():
        summary += " using regional structures"
    return summary.strip()

df["Summarized Vertical Coordination"] = df["Cleaned Practical Vertical Coordination"].apply(summarize_text)

# Step 3: Creating Themes
def assign_themes(summary):
    themes = []
    # Theme rules based on content
    if any(x in summary.lower() for x in ["assembly", "council", "secretariat"]):
        themes.append("Multi-Level Governance Structures")
    if any(x in summary.lower() for x in ["un agencies", "member states", "governments"]):
        themes.append("Collaboration with External Entities")
    if "regional" in summary.lower():
        themes.append("Regional and Local Implementation")
    if any(x in summary.lower() for x in ["technical", "assistance", "training"]):
        themes.append("Technical and Capacity-Building Support")
    if any(x in summary.lower() for x in ["director", "secretary", "leadership"]):
        themes.append("Strategic Leadership and Oversight")
    return ", ".join(themes)

# Frequency analysis for theme validation
def validate_themes(df):
    all_themes = df["Summarized Vertical Coordination"].apply(assign_themes).str.split(", ").explode()
    theme_counts = Counter(all_themes)
    print("Theme Frequency Counts:", theme_counts)

df["Themes"] = df["Summarized Vertical Coordination"].apply(assign_themes)
validate_themes(df)

# Step 4: Output Table
print("\nFinal Table:")
print(df[["Institution", "Cleaned Practical Vertical Coordination", "Summarized Vertical Coordination", "Themes"]].to_string(index=False))

# Save to CSV (optional)
df.to_csv("vertical_coordination_analysis.csv", index=False)
print("\nData saved to 'vertical_coordination_analysis.csv'")

Theme Frequency Counts: Counter({'Collaboration with External Entities': 3, 'Regional and Local Implementation': 2})

Final Table:
Institution                                                                                                                                                                                   Cleaned Practical Vertical Coordination                       Summarized Vertical Coordination                                                                  Themes
        IOC Vertical coordination within the IOC involves collaboration with various levels of institutions and authorities. The IOC, under UNESCO, engages with Member States, UN bodies, and other international organizations... (            Coordinates with UN agencies, Member States                                    Collaboration with External Entities
        FAO                      The FAO’s vertical coordination involves collaboration with various levels of governance and institutions, including the U

In [2]:
df

,Institution,Practical Vertical Coordination,Cleaned Practical Vertical Coordination,Summarized Vertical Coordination,Themes
0,IOC,Vertical coordination within the IOC involves ...,Vertical coordination within the IOC involves ...,"Coordinates with UN agencies, Member States",Collaboration with External Entities
1,FAO,The FAO’s vertical coordination involves colla...,The FAO’s vertical coordination involves colla...,Coordinates with UN agencies using regional st...,"Collaboration with External Entities, Regional..."
2,IMO,Vertical coordination within IMO involves coll...,Vertical coordination within IMO involves coll...,Coordinates with UN agencies using regional st...,"Collaboration with External Entities, Regional..."
